# Amazon OpenSearch Service에 Model(Remote Connector) 배포하기

Opensearch에 연결합니다.

In [1]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
import json

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=600,
)

# IAM Role에 OpenSearch의 ml_full_access 권한 부여하기

OpenSearch에 Model을 배포하기 위해서는 ml_full_access 권한이 필요합니다. 이를 위해서는 ml_full_access Role에 모델 배포 요청 Request를 Sign할 IAM Role ARN을 맵핑해주어야 합니다. 

먼저 ml_full_access의 Role Mapping을 확인합니다.

In [ ]:
import requests

security_url = "https://" + aos_host + "/_plugins/_security/api/rolesmapping/ml_full_access"

auth = (aos_credentials["username"], aos_credentials["password"])

create_role_mapping_body = {"users": [iam_role_arn]}

response = requests.get(
    security_url,
    auth=auth,
    # json=connector_payload,
)

json.loads(response.text)

현재 오직 `master` 사용자만이 ml_full_access 권한을 가지고 있을 것입니다. 여기에 `iam_role_arn`에 저장된 현재 IAM Role ARN을 추가해줍니다. 

In [2]:
모델 커넥터를 생성하는 요청을 보낼 때는 커넥터의 권한을 OpenSearch에게 넘겨줄 수 있는 PassRole 권한이 필요합니다. 하지만 OpenSearch의 Internal User(master)는 IAM 사용자가 아니므로 해당 권한을 설정할 수 없습니다. 따라서 credentials 정보를 `iam_role_arn`에 저장된 IAM Role의 정보로 변경합니다.

{'name': '4af019499bed',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'XqFSfdWsR_WtnvUSrxdb5Q',
 'version': {'distribution': 'opensearch',
  'number': '2.13.0',
  'build_type': 'tar',
  'build_hash': '7ec678d1b7c87d6e779fdef94e33623e1f1e2647',
  'build_date': '2024-03-26T00:02:39.659767978Z',
  'build_snapshot': False,
  'lucene_version': '9.10.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

# 임베딩 원격 모델 배포하기

In [ ]:
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region=region_name)

모델 커넥터 생성 요청을 위한 Base URL을 설정합니다. 

In [3]:
base_url = "https://" + aos_host + "/_plugins/_ml"
base_url

'https://localhost:9200/_plugins/_ml'

원격 모델 커넥터를 생성합니다.

In [7]:
import os
from dotenv import dotenv_values
import requests
env_vars = dotenv_values('.env_api')

# Bedrock Titan Embeddings G1 - Text Connector 생성

connector_payload = {
    "name": "Amazon Bedrock Connector: embedding",
    "description": "The connector to bedrock Titan embedding model",
    "version": 1,
    "protocol": "aws_sigv4",
    "parameters": {
        "region": region_name,
        "service_name": "bedrock",
        "model": "amazon.titan-embed-text-v2:0",
    },
    "credential": {"openAI_key": env_vars.get("OPENAI_API_KEY")},
    "actions": [
        {
            "action_type": "predict",
            "method": "POST",
            "url": "https://bedrock-runtime.${parameters.region}.amazonaws.com/model/${parameters.model}/invoke",
            "headers": {"content-type": "application/json", "x-amz-content-sha256": "required"},
            "request_body": '{ "inputText": "${parameters.inputText}" }',
            "pre_process_function": "connector.pre_process.bedrock.embedding",
            "post_process_function": "connector.post_process.bedrock.embedding",
        }
    ],
}

# Send the request
response = requests.post(
    base_url + "/connectors/_create",
    auth=auth,
    json=connector_payload,
)

# Print the response
print(response.text)

{"connector_id":"wmHLbpEBQH0FPvpr89Du"}


In [8]:
obj = json.loads(response.text)
connector_id = obj["connector_id"]
connector_id

'wmHLbpEBQH0FPvpr89Du'

원격 모델을 위한 모델 그룹을 생성합니다.

In [9]:
register_model_group_payload = {
    "name": "Bedrock Models for titan text v2",
    "description": "This is a model group for Amazon Bedrock remote model",
}

response = requests.post(
    base_url + "/model_groups/_register",
    auth=auth,
    json=register_model_group_payload,
)

# Print the response
print(response.text)

In [19]:
obj = json.loads(response.text)
try:
    model_group_id = obj["model_group_id"]
except:
    model_group_id = obj['error']['reason'].split(': ')[1][:-1]
print(model_group_id)

sGHLSpEBQH0FPvpr6syc


앞서 생성한 커넥터와 모델 그룹을 사용하여 모델을 등록합니다.

In [21]:
register_model_payload = {
    "name": "Bedrock text embedding model",
    "function_name": "remote",
    "model_group_id": model_group_id,
    "description": "This is Bedrock Titan Embeddings V2 - Text model",
    "connector_id": connector_id,
}

# Send the request
response = requests.post(
    base_url + "/models/_register",
    auth=auth,
    json=register_model_payload,
)

# Print the response
print(response.text)

{"task_id":"w2HPbpEBQH0FPvprOtAR","status":"CREATED","model_id":"xGHPbpEBQH0FPvprOtAr"}


In [22]:
obj = json.loads(response.text)
model_id = obj["model_id"]
print(model_id)

xGHPbpEBQH0FPvprOtAr


앞서 등록한 모델을 배포합니다.

In [23]:
deploy_model_payload = {}

# Send the request
response = requests.post(
    base_url + "/models/" + model_id + "/_deploy",
    auth=auth,
    json=deploy_model_payload,
)

# Print the response
print(response.text)

{"task_id":"xWHPbpEBQH0FPvpratBc","task_type":"DEPLOY_MODEL","status":"COMPLETED"}


모델이 잘 배포되었는지 테스트해봅니다.

In [24]:
base_url

'https://localhost:9200/_plugins/_ml'

In [25]:
predict_model_payload = {"parameters": {"inputText": "hello goodbye"}}

# Send the request
response = requests.post(
    base_url + "/models/" + model_id + "/_predict",
    auth=auth,
    json=predict_model_payload,
)

# Print the response
len(response.json()["inference_results"][0]["output"][0]["data"])

이후 테스트에 사용하기 위해 모델 아이디를 저장합니다.

In [ ]:
%store model_id

In [27]:
model_id

'xGHPbpEBQH0FPvprOtAr'